In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [1]:
cd yolov5

C:\Users\mahen\Intranet\Modele_IA\yolov5


In [4]:
!pip install -r requirements.txt

  Obtaining dependency information for gitpython>=3.1.30 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc457c723931fed3f/GitPython-3.1.43-py3-none-any.whl.metadata
  Obtaining dependency information for pillow>=10.3.0 from https://files.pythonhosted.org/packages/dc/83/1470c220a4ff06cd75fc609068f6605e567ea51df70557555c2ab6516b2c/pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for requests>=2.32.2 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4ff5221dc47127c80f8834b4bcd0cb36f6ba91c0b1d04a1233403/thop-0.1.1.post2209072238-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/78/18/7a2e56e2dc45a433dea9e1bf46a65e234294c9c470ccb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


   -------------------- ----------------- 111.4/203.1 MB 281.6 kB/s eta 0:05:26
   -------------------- ----------------- 111.4/203.1 MB 282.3 kB/s eta 0:05:25
   -------------------- ----------------- 111.5/203.1 MB 283.5 kB/s eta 0:05:24
   -------------------- ----------------- 111.5/203.1 MB 283.3 kB/s eta 0:05:24
   -------------------- ----------------- 111.5/203.1 MB 283.8 kB/s eta 0:05:23
   -------------------- ----------------- 111.6/203.1 MB 286.1 kB/s eta 0:05:21
   -------------------- ----------------- 111.6/203.1 MB 286.4 kB/s eta 0:05:20
   -------------------- ----------------- 111.6/203.1 MB 286.5 kB/s eta 0:05:20
   -------------------- ----------------- 111.6/203.1 MB 287.2 kB/s eta 0:05:19
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------

In [2]:
import torch
import cv2
import numpy as np
from facenet_pytorch import InceptionResnetV1
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from torchvision import transforms

In [3]:
def load_image(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)
    return image

In [4]:
def load_yolo_model():
    # Charger le modèle YOLOv5 small pré-entraîné
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    return model


In [5]:
def detect_faces_yolo(model, image):
    # Effectuer la détection des objets
    results = model(image)
    
    # Extraire les boîtes englobantes des objets détectés (en tant que tableau numpy)
    detections = results.xyxy[0].numpy()
    
    return detections

In [6]:
def detect_landmarks_mtcnn(face_region):
    detector = MTCNN()
    results = detector.detect_faces(face_region)
    if results:
        landmarks = []
        for result in results:
            if 'keypoints' in result:
                landmarks.append(result['keypoints'])  
        return landmarks
    return None

In [7]:
def align_face(image, left_eye, right_eye):
    left_eye_center = np.array(left_eye)
    right_eye_center = np.array(right_eye)
    delta_x = right_eye_center[0] - left_eye_center[0]
    delta_y = right_eye_center[1] - left_eye_center[1]
    angle = np.arctan2(delta_y, delta_x) * 180 / np.pi
    eyes_center = (float((left_eye_center[0] + right_eye_center[0]) // 2),
                   float((left_eye_center[1] + right_eye_center[1]) // 2))
    rotation_matrix = cv2.getRotationMatrix2D(eyes_center, angle, scale=1)
    aligned_face = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
    return aligned_face

In [8]:
def draw_rectangle_and_points(image, x1, y1, x2, y2, landmarks=None):
    # Dessiner un rectangle autour du visage
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Couleur verte (0, 255, 0)
    
    # Si les points de repère sont fournis, dessiner des points sur les yeux, le nez et la bouche
    if landmarks:
        # Ajuster les coordonnées des points de repère en ajoutant x1 et y1
        for key, point in landmarks.items():
            adjusted_point = (point[0] + int(x1), point[1] + int(y1))  # Ajustement des coordonnées
            cv2.circle(image, adjusted_point, radius=3, color=(0, 0, 255), thickness=-1)  # Couleur rouge pour les points

    return image

In [9]:
def display_image(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [17]:
def process_image_yolo_mtcnn(image_path):
    # Charger l'image
    image = load_image(image_path)

    # Charger le modèle YOLOv5
    model = load_yolo_model()

    # Détecter les visages dans l'image avec YOLO
    detections = detect_faces_yolo(model, image)

    # Charger le modèle InceptionResnetV1 pour l'extraction d'embeddings
    resnet = InceptionResnetV1(pretrained='vggface2').eval()

    # Traiter chaque visage détecté
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == 0:  # Vérifier que la classe correspond bien à "personne"
            # Extraire la région du visage détecté
            face_region = image[int(y1):int(y2), int(x1):int(x2)]

            # Détecter les points de repère dans la région du visage avec MTCNN
            landmarks_list = detect_landmarks_mtcnn(face_region)

            if landmarks_list:
                for landmarks in landmarks_list:
                    # Récupérer les positions des yeux, du nez et des lèvres
                    left_eye = landmarks['left_eye']
                    right_eye = landmarks['right_eye']
                    nose = landmarks['nose']

                    # Aligner le visage en fonction des yeux
                    aligned_face = align_face(face_region, left_eye, right_eye)

                    # Redimensionner l'image alignée à la taille requise pour InceptionResnetV1 (160x160)
                    resized_face = cv2.resize(aligned_face, (160, 160))

                    # Convertir l'image en tensor et ajouter une dimension batch
                    face_tensor = torch.tensor(np.transpose(resized_face, (2, 0, 1))).float().unsqueeze(0)

                    # Extraire les embeddings
                    embeddings = resnet(face_tensor)

                    # Afficher l'image avec le rectangle et les points de repère
                    image_with_rectangle_and_points = draw_rectangle_and_points(image, x1, y1, x2, y2, landmarks)
                    display_image(image_with_rectangle_and_points)

                    # Afficher le visage aligné
                    display_image(aligned_face)

                    print(f"Embeddings générés : {embeddings.detach().numpy()}")

            else:
                print("Points de repère non détectés pour ce visage.")
        else:
            print(f"Objet détecté, mais ce n'est pas un visage (classe {cls})")

In [11]:
image = "C:/Users/mahen/Intranet/Modele_IA/images/profil_picture/selena_profile_picture.jpg"

In [12]:
images = "C:/Users/mahen/Intranet/Modele_IA/images/picture/selena_picture_1.jpg"

In [18]:
process_image_yolo_mtcnn(image)

Using cache found in C:\Users\mahen/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-20 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Embeddings générés : [[   0.059515   -0.035364    0.051321  -0.0090521    0.051175   -0.067479    0.013195   -0.059969    0.020651   -0.032917    -0.06124   -0.073359   -0.080372   -0.033789    0.090747   -0.014903  -0.0064603  -0.0043214    0.015266    0.069227    0.029732  -0.0019071   -0.034608  -0.0076145    -0.01779     0.01771
     0.060037   -0.024375    0.016333    -0.11183   -0.029646    0.077222   -0.011963     0.04054    0.059493    0.015257   -0.014394    0.037753   -0.039667    0.040027    0.016197    -0.02453   -0.033372   -0.088511  -0.0082957   -0.044419    0.051927   -0.014219   -0.030742    0.039299     0.11342    0.047533
   -0.0044901     0.07888    0.026474   -0.057947    0.051406    0.040994    0.027959    0.046284   -0.029197    -0.10256    0.042842   -0.044903   0.0064497    0.015831    0.036588   0.0096164   -0.015815    -0.08151   0.0092872   -0.074293  -0.0097582    -0.02792 -6.2098e-05    0.014618     0.08145    0.020406
    -0.038958    0.039918  -0.0067264

In [19]:
process_image_yolo_mtcnn(images)

Using cache found in C:\Users\mahen/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-20 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Embeddings générés : [[   0.057337    -0.02943    0.031904 -0.00015989     0.04099   -0.073983  -0.0021853   -0.023898    0.022219    -0.04305   -0.080322   -0.070645   -0.078249   -0.015664    0.084959  -0.0098595   -0.010186  -0.0036523     0.02578    0.053265    0.028046  -0.0065198     -0.0383   -0.018327   -0.006599    0.018977
     0.068765   -0.015624    0.042544    -0.11119   -0.046561    0.053674  -0.0036655    0.042912    0.054383    0.026175   -0.018561    0.049538   -0.033726    0.033764    0.017778  -0.0079648   -0.026988   -0.097361   -0.018417   -0.053267    0.024881  -0.0036319   -0.060053    0.033677    0.083909    0.054002
     0.013062    0.063965   -0.010115   -0.063362    0.053011    0.047623    0.028331    0.040003   -0.032273    -0.10261    0.044464   -0.029713  -0.0076274    0.012072    0.060177    0.018439   -0.015346   -0.091366   0.0066855    -0.10656    0.010021   -0.021764  -0.0067034   0.0043468    0.077229    0.034297
    -0.044703    0.034124   0.0047595

Embeddings générés : [[   0.075831   -0.055639    0.033915  -0.0022433    0.018497   -0.064122   -0.024072   -0.028021    0.010919   -0.037017   -0.064901   -0.034302   -0.075956   -0.015937    0.075658   -0.046506  -0.0056487  -0.0020331    0.019012    0.043526    0.032753    0.029566    -0.02404   -0.029097   -0.020703    0.035767
     0.063046   -0.028044    0.026656    -0.10636   -0.010762    0.068262    0.029379    0.045706    0.054376       0.022   -0.059351    0.053157   -0.041069    0.049802   0.0017977   -0.040127   -0.030441   -0.096675   -0.035957   -0.064834    0.031494  -0.0055547   -0.066351    0.022902     0.10282    0.069717
      0.03766    0.082617   0.0065883   -0.055291    0.051727    0.074465   0.0062022    0.043402   -0.028435   -0.082711    0.036466   -0.032949    0.017427    0.024672     0.06088   0.0060635 -0.00060765   -0.080567    0.034354   -0.072881   -0.005274   -0.024657   -0.014095    0.016152    0.070307    0.010311
    -0.039609    0.051263  -0.0029212

Embeddings générés : [[   0.067643   -0.044753    0.042336   0.0015123     0.02027   -0.076713    -0.01309   -0.045675    0.023462   -0.041636   -0.061893   -0.043199   -0.080074   -0.016115    0.067478   -0.037087   -0.015356   -0.002235     0.02518    0.042026    0.027625    0.014454   -0.023948    -0.02725   -0.022379    0.030994
      0.05992   -0.036689    0.025839    -0.10466   -0.019774    0.068639    0.035691    0.038821    0.054299     0.02438   -0.043751    0.057822   -0.035899    0.051073  -0.0015843   -0.046959   -0.037637   -0.094091   -0.038604   -0.059955    0.023663    0.009642   -0.057294    0.037398     0.10348    0.067138
     0.017126    0.078816   0.0075441   -0.059936    0.066013    0.069699   0.0018472    0.046239   -0.039866   -0.088108    0.040679    -0.03293    0.011594    0.017056    0.054696    0.011753   -0.011797   -0.091752    0.038301   -0.062024  -0.0040762   -0.021252   -0.020466   0.0012865    0.068047    0.015401
    -0.045345    0.058978    -0.01264

Points de repère non détectés pour ce visage.
Embeddings générés : [[    0.05933   -0.049229    0.033936   -0.011523    0.065993   -0.081238   0.0055896   -0.054869    0.014455  -0.0065785   -0.092521   -0.051043   -0.085738  -0.0040695    0.074824   -0.013211   0.0022209   0.0049801    0.038548    0.076447    0.053662    0.010136    -0.04258 -0.00018086    -0.01085    0.028712
     0.067447    -0.00719    0.044717    -0.12491   -0.046614    0.035119   0.0035147    0.032399    0.084968  -0.0073645   -0.051373    0.038504   -0.020536    0.049154  -0.0048194   -0.014464   -0.041423    -0.09446   -0.034963   -0.037127    0.025585   -0.035155   -0.045009    0.062761    0.079088    0.036474
    0.0077135    0.067628   -0.008032   -0.083568    0.060005    0.038678    0.015335    0.024915   -0.015921   -0.091261    0.046091   -0.021596  -0.0092797   0.0018078    0.068463  -0.0038851   -0.021284   -0.076891   0.0060659   -0.099633  -0.0010753   -0.001053   -0.037415    0.034362    0.089438    

In [33]:
import torch
print(torch.__version__)

2.5.0+cpu


In [16]:
def display_image(image):
    cv2.imshow("Image avec détection de visages", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [15]:
!pip install --upgrade backports.tarfile

  Obtaining dependency information for backports.tarfile from https://files.pythonhosted.org/packages/b9/fa/123043af240e49752f1c4bd24da5053b6bd00cad78c2be53c0d1e8b975bc/backports.tarfile-1.2.0-py3-none-any.whl.metadata


In [16]:
!pip install --upgrade setuptools
!pip install --upgrade pkg_resources


ERROR: Could not find a version that satisfies the requirement pkg_resources (from versions: none)
ERROR: No matching distribution found for pkg_resources


In [20]:
image2="C:/Users/mahen/Intranet/Modele_IA/images/profil_picture/selena_profile_picture2.jpg"

In [22]:
process_image_yolo_mtcnn(image2)

Using cache found in C:\Users\mahen/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-20 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Embeddings générés : [[   0.064083   -0.060447    0.031066   -0.011665    0.041465   -0.063995   0.0044914   -0.059873    0.015228    -0.02605   -0.071699   -0.036739   -0.081963   -0.017058    0.083477   -0.026953  -0.0088167  -0.0028205    0.026105    0.060523    0.041214    0.016596   -0.020803  -0.0081868   -0.014783    0.026588
     0.058886   -0.037397    0.013339    -0.11396  -0.0084031    0.065088    0.015961     0.03884     0.06801    0.020202   -0.021832    0.056784   -0.031549     0.04567   0.0091206   -0.051667   -0.048086    -0.10041   -0.041941   -0.070685    0.038282    -0.01896   -0.051535    0.045082     0.10391    0.048914
    0.0080754    0.083207   0.0049672   -0.070058    0.054313    0.059968    0.014544    0.036202   -0.026902   -0.080995    0.031843   -0.029789   0.0099386    0.014239    0.072972  -0.0081607  0.00089922    -0.07784    0.017872   -0.084531  -0.0058503   -0.015345   -0.030402    0.024852    0.078858   0.0052038
    -0.025841    0.054313  -0.0052541